In [100]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
import camelot

In [101]:
r = requests.get('https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Situationsberichte/Gesamt.html')

In [102]:
r

<Response [200]>

In [103]:
soup = BeautifulSoup(r.text, 'lxml')

### Link finden 

In [104]:
links = soup.findAll('a', {'class': 'themenLink'})
link = links[0].get('href')
link

'/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Situationsberichte/Sept_2020/2020-09-22-de.pdf?__blob=publicationFile'

In [105]:
base = 'https://www.rki.de/'
url = base + link
url

'https://www.rki.de//DE/Content/InfAZ/N/Neuartiges_Coronavirus/Situationsberichte/Sept_2020/2020-09-22-de.pdf?__blob=publicationFile'

In [106]:
import pdftotext
import io

In [107]:
r = requests.get(url)
f = io.BytesIO(r.content)

In [108]:
# Load your PDF
pdf = pdftotext.PDF(f)

In [109]:
# How many pages?
print(len(pdf))

16


In [110]:
# Iterate over all the pages
for page in pdf:
    page_search = re.search('4-Tage-R-Wert', page)
    if page_search:
        text = page

In [111]:
text

'13                                                                                                  COVID-19-Lagebericht vom 22.09.2020       13\n   Im SK München, wo mehrere Ausbruchsgeschehen in verschiedenen Settings für die erhöhte Inzidenz\n   ursächlich sind, liegt die Inzidenz heute wieder knapp unter 50/100.000 Einwohner. In den nächsten\n   Tagen sollen weitere Eindämmungsmaßnahmen verpflichtend werden.\n   COVID-19-Fälle treten besonders in Zusammenhang mit Feiern im Familien- und Freundeskreis auf. Der\n   Anteil der Reiserückkehrer unter den Fällen geht zurück.\n   Es werden wieder vermehrt COVID-19-bedingte Ausbrüche in Alten- und Pflegeheimen gemeldet; weitere\n   Ausbrüche gibt es in Krankenhäusern, Einrichtungen für Asylbewerber und Geflüchtete,\n   Gemeinschaftseinrichtungen, verschiedenen beruflichen Settings sowie in Zusammenhang mit religiösen\n   Veranstaltungen.\n   Schätzung der Fallzahlen unter Berücksichtigung des Verzugs (Nowcasting) und der\n   Reproduktions

In [112]:
r = re.compile('\d,\d{2}')

In [113]:
text = text.replace(' ', '')

In [114]:
text

'13COVID-19-Lageberichtvom22.09.202013\nImSKMünchen,womehrereAusbruchsgescheheninverschiedenenSettingsfürdieerhöhteInzidenz\nursächlichsind,liegtdieInzidenzheutewiederknappunter50/100.000Einwohner.Indennächsten\nTagensollenweitereEindämmungsmaßnahmenverpflichtendwerden.\nCOVID-19-FälletretenbesondersinZusammenhangmitFeiernimFamilien-undFreundeskreisauf.Der\nAnteilderReiserückkehrerunterdenFällengehtzurück.\nEswerdenwiedervermehrtCOVID-19-bedingteAusbrücheinAlten-undPflegeheimengemeldet;weitere\nAusbrüchegibtesinKrankenhäusern,EinrichtungenfürAsylbewerberundGeflüchtete,\nGemeinschaftseinrichtungen,verschiedenenberuflichenSettingssowieinZusammenhangmitreligiösen\nVeranstaltungen.\nSchätzungderFallzahlenunterBerücksichtigungdesVerzugs(Nowcasting)undder\nReproduktionszahl(R)\nDieReproduktionszahlRbezeichnetdieAnzahlderPersonen,dieimDurchschnittvoneinemFall\nangestecktwerden.DieselässtsichnichtausdenMeldedatenablesen,sondernnurdurchstatistische\nVerfahrenschätzen,zumBeispielaufderBasisdesNo

In [115]:
matches = list(r.findall(text))
matches

['0,92', '0,99', '0,73', '1,12', '0,89', '1,10']

In [116]:
get_date = re.compile('(COVID-19-Lageberichtvom)(\d{2}\.\d{2}\.\d{4})')

In [117]:
date_search = re.search('(COVID-19-Lageberichtvom)(\d{2}\.\d{2}\.\d{4})', text)

if date_search:
    date = date_search.group(2)

In [118]:
date = date[:5]

In [119]:
my_dict = {'Datum': date, 
           '7-Tage-oben': matches[5], '7-Tage-Mitte': matches[1], '7-Tage-unten': matches[4], 
           '4-Tage-oben': matches[3], '4-Tage-Mitte': matches[0], '4-Tage-unten': matches[2]}

In [120]:
my_dict

{'Datum': '22.09',
 '7-Tage-oben': '1,10',
 '7-Tage-Mitte': '0,99',
 '7-Tage-unten': '0,89',
 '4-Tage-oben': '1,12',
 '4-Tage-Mitte': '0,92',
 '4-Tage-unten': '0,73'}

In [121]:
df = pd.DataFrame(my_dict, index=[0] )

In [122]:
df

,Datum,7-Tage-oben,7-Tage-Mitte,7-Tage-unten,4-Tage-oben,4-Tage-Mitte,4-Tage-unten
0,22.09,"1,10","0,99","0,89","1,12","0,92","0,73"
